***GENERATED CODE FOR clustering PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'user_id', 'transformation_label': 'String Indexer'}], 'feature': 'user_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '14', 'mean': '', 'stddev': '', 'min': 'user31', 'max': 'user33', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'user_id'}, {'feature_label': 'user_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('user_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'hotel_name', 'transformation_label': 'String Indexer'}], 'feature': 'hotel_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '14', 'mean': '', 'stddev': '', 'min': '128 Bangkok', 'max': 'Villa Phra Sumen Bangkok', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'hotel_name'}, {'feature_label': 'hotel_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('hotel_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'trip_type', 'transformation_label': 'String Indexer'}], 'feature': 'trip_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '14', 'mean': '', 'stddev': '', 'min': 'business', 'max': 'leisure', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'trip_type'}, {'feature_label': 'trip_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('trip_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'companion', 'transformation_label': 'String Indexer'}], 'feature': 'companion', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '14', 'mean': '', 'stddev': '', 'min': 'couple', 'max': 'solo', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'companion'}, {'feature_label': 'companion', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('companion')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'destination', 'transformation_label': 'String Indexer'}], 'feature': 'destination', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '14', 'mean': '', 'stddev': '', 'min': 'bangkok', 'max': 'bangkok', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'destination'}, {'feature_label': 'destination', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('destination')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        autokmeans_obj = kmeans(df=spark_DF, TrainingPercent=trainingPercent)
        [autokmeans_obj]


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run clusteringHooks.ipynb
try:
	#sourcePreExecutionHook()

	hotelrecommender = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Hotel_Recommender.csv', 'filename': 'Hotel_Recommender.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://10.0.5.54', 'port': '50070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(hotelrecommender)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run clusteringHooks.ipynb
try:
	#transformationPreExecutionHook()

	clusteringautofe = TransformationMain.run(hotelrecommender,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "user_id", "transformation_label": "String Indexer"}], "feature": "user_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "14", "mean": "", "stddev": "", "min": "user31", "max": "user33", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "user_id"}, {"transformationsData": [{"feature_label": "hotel_name", "transformation_label": "String Indexer"}], "feature": "hotel_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "14", "mean": "", "stddev": "", "min": "128 Bangkok", "max": "Villa Phra Sumen Bangkok", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "hotel_name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "14", "mean": "8.57", "stddev": "0.47", "min": "8.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "14", "mean": "3268.43", "stddev": "2514.96", "min": "1215", "max": "9534", "missing": "0"}, "updatedLabel": "price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "near_station", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "14", "mean": "0.22", "stddev": "0.07", "min": "0.131147541", "max": "0.307692308", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "near_station"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "restaurant", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "14", "mean": "0.32", "stddev": "0.07", "min": "0.243902439", "max": "0.454545455", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "restaurant"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "entertain", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "14", "mean": "0.11", "stddev": "0.07", "min": "0.036585366", "max": "0.240506329", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "entertain"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "shopping_mall", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "14", "mean": "0.16", "stddev": "0.06", "min": "0.045454545", "max": "0.243902439", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "shopping_mall"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "convenience_store", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "14", "mean": "0.23", "stddev": "0.05", "min": "0.127659574", "max": "0.301886792", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "convenience_store"}, {"transformationsData": [{"feature_label": "trip_type", "transformation_label": "String Indexer"}], "feature": "trip_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "14", "mean": "", "stddev": "", "min": "business", "max": "leisure", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "trip_type"}, {"transformationsData": [{"feature_label": "companion", "transformation_label": "String Indexer"}], "feature": "companion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "14", "mean": "", "stddev": "", "min": "couple", "max": "solo", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "companion"}, {"transformationsData": [{"feature_label": "destination", "transformation_label": "String Indexer"}], "feature": "destination", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "14", "mean": "", "stddev": "", "min": "bangkok", "max": "bangkok", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "destination"}]}))

	#transformationPostExecutionHook(clusteringautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run clusteringHooks.ipynb
try:
	#mlPreExecutionHook()

	clusteringclustering = Clustering.run(clusteringautofe,spark,json.dumps( {"autoClustering": 1, "originalfile": "/FileStore/platform/uploadedSourceFiles/Hotel_Recommender.csv", "model": "Kmeans", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(clusteringclustering)

except Exception as ex: 
	logging.error(ex)
